# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)

In [2]:
# 2) seu código aqui
sinasc.isna().sum().sort_values(ascending=False)

DTRECORIGA    27028
CODANOMAL     26814
IDADEPAI      19421
SERIESCMAE    12710
DTULTMENST    10072
              ...  
PESO              0
DTCADASTRO        0
NUMEROLOTE        0
VERSAOSIST        0
ORIGEM            0
Length: 69, dtype: int64

In [3]:
# 3) seu código aqui
colunas = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
sinasc_df = sinasc[colunas]
sinasc_df

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [4]:
# 4) seu código aqui
sinasc_df.isna().sum().sort_values(ascending=False)
sinasc_df['APGAR5'].dropna(inplace=True)

C:\Users\thomas.costa\AppData\Local\Temp\ipykernel_40372\671845696.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_df['APGAR5'].dropna(inplace=True)


In [5]:
# 5) seu código aqui
sinasc_df[['ESTCIVMAE','CONSULTAS']].fillna(9)

,ESTCIVMAE,CONSULTAS
0,5.0,4
1,2.0,4
2,9.0,4
3,5.0,3
4,2.0,4
...,...,...
27023,2.0,4
27024,1.0,4
27025,2.0,4
27026,2.0,4


In [6]:
# 6) Seu código aqui
sinasc_df['QTDFILVIVO'].fillna(0)

0        0.0
1        1.0
2        2.0
3        0.0
4        1.0
        ... 
27023    1.0
27024    0.0
27025    0.0
27026    1.0
27027    1.0
Name: QTDFILVIVO, Length: 27028, dtype: float64

In [7]:
# 7) seu código aqui
#Relativo a coluna GESTACAO e GRAVIDEZ para os casos de NaN foi adotado o critério do valor mais recorrente
maior_repeticoes_gravidez = sinasc_df['GRAVIDEZ'].value_counts().idxmax()
maior_repeticoes_gestacao = sinasc_df['GESTACAO'].value_counts().idxmax()
sinasc_df['GRAVIDEZ'].fillna(maior_repeticoes_gravidez)
sinasc_df['GESTACAO'].fillna(maior_repeticoes_gestacao)
#Quanto a coluna APGAR5 utilizaremos para os valores vazios o valor 0 quando não houver valor
sinasc_df['APGAR5'].fillna(0)
#Quanto a coluna ESCMAE utilizaremos para os valores vazios o valor "Ignorado" devido ao dado ser texto no dataframe
sinasc_df['ESCMAE'].fillna('Ignorado')

sinasc_df.isna().sum().sort_values(ascending=False)

QTDFILVIVO    1573
GESTACAO      1232
ESTCIVMAE      317
ESCMAE         312
APGAR5         103
GRAVIDEZ        79
LOCNASC          0
IDADEMAE         0
CONSULTAS        0
dtype: int64

In [8]:
# 8) seu código aqui
# Entre 8 e 10 está em uma faixa 'normal'. 
# Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
# Entre 4 e 5 significa 'asfixia moderada'.
# Entre 0 e 3 significa 'asfixia severa'.  
sinasc_df.loc[sinasc_df['APGAR5'] <= 3, 'APGAR5_CAT'] = 'asfixia severa'
sinasc_df.loc[(sinasc_df['APGAR5'] >= 4) & (sinasc_df['APGAR5'] <= 5),
           'APGAR5_CAT'] = 'asfixia moderada'
sinasc_df.loc[(sinasc_df['APGAR5'] >= 6) & (sinasc_df['APGAR5'] <= 7),
           'APGAR5_CAT'] = 'asfixia leve'
sinasc_df.loc[(sinasc_df['APGAR5'] >= 8), 'APGAR5_CAT'] = 'normal'

sinasc_df

C:\Users\thomas.costa\AppData\Local\Temp\ipykernel_40372\2709975988.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_df.loc[sinasc_df['APGAR5'] <= 3, 'APGAR5_CAT'] = 'asfixia severa'


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,APGAR5_CAT
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal


In [9]:
# 9) seu código aqui
sinasc_df.columns = ['loc_nasc','idade_mae','est_civ_mae','esc_mae','qtd_fil_vivo','gestacao','gravidez','consultas','apgar_5','apgar_5_cat']
sinasc_df.columns

Index(['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo',
       'gestacao', 'gravidez', 'consultas', 'apgar_5', 'apgar_5_cat'],
      dtype='object')